In [1]:
#pip install --upgrade pip


In [2]:
#pip install html5lib

In [3]:
#pip install selenium

In [4]:
#pip install bs4

In [1]:
import pandas as pd
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import re

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By



In [3]:


driver = webdriver.Chrome()
driver.get("https://www.foxnews.com/category/shows/tucker-carlson-tonight/transcript")
button = driver.find_element(By.XPATH, "//div[@class='button load-more js-load-more']") 
x = 0
for x in range(100):
    button.click()    
    time.sleep(1)
    x+=1
    print(x)
page_source = driver.page_source # save the source to feed to beautiful soup 
print("saved")

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
saved


In [4]:
soup = BeautifulSoup(page_source)


In [5]:
url_soup = soup.find_all('a', href=re.compile('/opinion/.*'))

In [6]:
#url_soup

In [7]:
url_list = []
prefix = 'https://www.foxnews.com/'
for link in url_soup: 
    suffix = re.findall('\"\/(.*?)\"', str(link))
    if suffix:
        url_list.append(prefix + suffix[0])
    

In [8]:
url_list[52]

'https://www.foxnews.com/opinion/tucker-carlson-zelenskyys-visit-represented-bipartisan-masochism'

In [18]:
# verifying for one instance
url = url_list[200]
#url = 'https://www.foxnews.com/opinion/tucker-carlson-faa-travel-pete-buttigieg'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')
title = soup.title.string

# #results = soup.find(id='wrapper')
# #body = soup.find_all('p')
# #text = body[1].text

a_tag = soup.find_all('a')
for tag in a_tag:
    tag.decompose()

copy_tag = soup.find_all('p', class_ = "copyright")
for tag in copy_tag:
    tag.decompose()
    
span_tag = soup.find_all('span')
for tag in span_tag:
    tag.decompose()
    
bold_tag = soup.find_all('strong') 
for tag in bold_tag:
    tag.decompose()
    
italic_tag=soup.find_all('i') 
for tag in italic_tag:
    tag.decompose()

other_tag = soup.find_all('p', class_ ="dek")
for tag in other_tag:
    tag.decompose()
    
other_tag2 = soup.find_all('p', class_ ="subscribed hide")
for tag in other_tag2:
    tag.decompose()
    
other_tag3 = soup.find_all('p', class_ ="success hide") 
for tag in other_tag3:
    tag.decompose()



final = soup.find_all('p')
# text=''
# print(final)
finaltext = ''
for p in final:

    try:
        finaltext = finaltext + p.text
        finaltext = finaltext.replace(u'\xa0', u' ')

    except:
        continue
finaltext
# #finaltext = ' '.join(text)
# #text
# # #soup.find_all('p')
# #soup.find('time').text

'Fox News host Tucker Carlson weighs in on the power George Soros has all over the world on \'Tucker Carlson Tonight.\'When a Hungarian financier called  shorted the English pound 30 years ago, back in 1992, the British government fought back. It did everything it could to preserve the value of its national currency. British officials raised interest rates all the way to 15%. The Bank of England then sold $40 billion of foreign currency reserves in a desperate effort to prop up the pound. In the end, of course, it didn\'t work. The Bank of England collapsed. Total cost to the population of Great Britain? Well, that\'s hard to measure, but it was at least 3 billion pounds. This at a time when a pound was worth nearly 2-and-a-half dollars. It\'s now at $1.20, which tells you a lot. The U.K. got poorer, but not George Soros. His fund made off with a billion dollars in profit — a billion dollars for creating nothing, only destroying things.  In the years since, Soros has become richer stil

In [19]:

full_df = None
dfs = []

for n, url in enumerate(url_list):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    time = soup.find('time').text
    title = soup.title.string

    a_tag = soup.find_all('a')
    for tag in a_tag:
        tag.decompose()

    copy_tag = soup.find_all('p', class_ = "copyright")
    for tag in copy_tag:
        tag.decompose()

    span_tag = soup.find_all('span')
    for tag in span_tag:
        tag.decompose()

    bold_tag = soup.find_all('strong') 
    for tag in bold_tag:
        tag.decompose()

    italic_tag=soup.find_all('i') 
    for tag in italic_tag:
        tag.decompose()

    other_tag = soup.find_all('p', class_ ="dek")
    for tag in other_tag:
        tag.decompose()

    other_tag2 = soup.find_all('p', class_ ="subscribed hide")
    for tag in other_tag2:
        tag.decompose()

    other_tag3 = soup.find_all('p', class_ ="success hide") 
    for tag in other_tag3:
        tag.decompose()


    final = soup.find_all('p')
    # text=''
    # print(final)
    finaltext = ''
    for p in final:

        try:
            finaltext = finaltext + p.text
            finaltext = finaltext.replace(u'\xa0', u' ')
        except:
            continue
    
    data={'url': [url],'timestamp': [time], 'title': [title], 'text': [finaltext]}
    df = pd.DataFrame(data)
    dfs.append(df)


In [20]:
full_df

In [21]:
full_df = pd.concat(dfs).reset_index(drop=True)

In [22]:
full_df.iloc[19,2]

'TUCKER CARLSON: If you really want to understand how powerful Big Pharma is, consider this | Fox News'

In [23]:
no_duplicates=full_df.drop_duplicates().reset_index(drop=True)

In [24]:
len(no_duplicates)

647

In [25]:
no_duplicates

,url,timestamp,title,text
0,https://www.foxnews.com/opinion/tucker-carlson...,"February 9, 2023 10:19pm EST",TUCKER CARLSON: Self-defense is becoming illeg...,WARNING: Graphic footage—Fox News host Tucker ...
1,https://www.foxnews.com/opinion/tucker-carlson...,"February 8, 2023 10:25pm EST",TUCKER CARLSON: The lies 'just kept coming' du...,Fox News host Tucker Carlson reacts to Presid...
2,https://www.foxnews.com/opinion/tucker-carlson...,"February 7, 2023 10:51pm EST",TUCKER CARLSON: America's problems are fixable...,Fox News host Tucker Carlson breaks down the t...
3,https://www.foxnews.com/opinion/tucker-carlson...,"February 6, 2023 9:33pm EST",TUCKER CARLSON: The people who owe you the tru...,Fox News host Tucker Carlson calls out the Bid...
4,https://www.foxnews.com/opinion/tucker-carlson...,"February 3, 2023 9:59pm EST",TUCKER CARLSON: Biden administration's stance ...,Fox News host Tucker Carlson takes on the Bid...
...,...,...,...,...
642,https://www.foxnews.com/opinion/tucker-carlson...,"November 19, 2019 1:30pm EST","Tucker Carlson: Media, federal government toug...",Stone found guilty on all seven counts; Epstei...
643,https://www.foxnews.com/opinion/tucker-carlson...,"November 15, 2019 1:09pm EST",Tucker Carlson: Trump impeachment inquiry will...,Media goes into overdrive on impeachment heari...
644,https://www.foxnews.com/opinion/tucker-carlson...,"November 14, 2019 9:24am EST",Tucker Carlson: It's never been clear what Dem...,First public impeachment hearings take place.I...
645,https://www.foxnews.com/opinion/tucker-carlson...,"November 13, 2019 10:13am EST",Tucker Carlson: The end of free speech – What ...,Liberals working to crack down on the First Am...


In [27]:
no_duplicates.to_csv('Tucker_transcripts_2.10.23.tsv', sep="\t")